<a href="https://colab.research.google.com/github/harshitha020505/MLLab1/blob/main/MLLab3-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv("titanic_train.csv")
print(df)

In [ ]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].mean(), inplace=True)


In [ ]:
df.ffill()
print(df.isnull().sum)


In [ ]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df.columns if df[col].isnull().sum()>0]
X=df.drop('Survived',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df['Survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

In [ ]:
#quasi-constant features
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

In [ ]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

In [ ]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])  # Only numeric columns
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]
                   ) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df, 0.9)
corr_final = set(corr_features)
print(len(corr_final))
X_train.drop(columns=corr_final, inplace=True)
X_test.drop(columns=corr_final, inplace=True)

In [ ]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import pandas as pd

X = df.drop('Survived', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)


In [ ]:
#CHI SQUARE TEst
from scipy.stats import chi2_contingency
result=[]
for feature in X_train.columns:
  crosstab=pd.crosstab(X_train[feature],df['Survived'])
  chi2_stat,p_val,dof,expected=chi2_contingency(crosstab)
  if p_val<0.05:
    result.append(feature)
print(result)
X_train.drop(columns=feature, inplace=True)
X_test.drop(columns=feature, inplace=True)

In [ ]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

In [ ]:
#forward Selection
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,scoring='roc_auc' or 'accuracy',cv=3)


In [ ]:
sfs1=sfs1.fit(X_train,y_train)


In [ ]:
print(sfs1.k_score_)
print(sfs1.k_feature_idx_)
print(sfs1.k_feature_names_)

In [ ]:
#Backward Selection
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,scoring='roc_auc' or 'accuracy',cv=3)
sfs1=sfs1.fit(np.array(X_train),y_train)
print(sfs1.k_score_)
print(sfs1.k_feature_idx_)
print(sfs1.k_feature_names_)

In [ ]:
#exhastive feature selection
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestClassifier
efs1=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=3)
efs1=efs1.fit(X_train,y_train)
print(efs1.best_idx_)
print(efs1.best_score_)
print(efs1.best_feature_names_)


In [ ]:
# Principal Component Analysis And Linear Discriminant Analysis
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


X["Sex"] = LabelEncoder().fit_transform(X["Sex"])
X["Embarked"] = LabelEncoder().fit_transform(X["Embarked"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Explained variance ratio by PCA:", pca.explained_variance_ratio_)
print("Shape after PCA:", X_pca.shape)
print("Shape after LDA:", X_lda.shape)


In [ ]:
import pandas as pd
df1=pd.read_csv("Santander Customer Satisfaction_train.csv")
print(df1)

In [ ]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df1.columns if df1[col].isnull().sum()>0]
X=df1.drop('TARGET',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df1['TARGET']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

In [ ]:
#quasi-constant features
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

In [ ]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

In [ ]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df1, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')


In [ ]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df1 = df1.dropna(subset=['TARGET'])
X = df1.drop('TARGET', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)


In [ ]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

In [ ]:
#forward Selection
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,scoring='roc_auc' or 'accuracy',cv=3)
sfs1=sfs1.fit(X_train,y_train)
print(sfs1.k_score_)
print(sfs1.k_feature_idx_)
print(sfs1.k_feature_names_)


In [ ]:
#Backward Selection
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,scoring='roc_auc' or 'accuracy',cv=3)
sfs1=sfs1.fit(np.array(X_train),y_train)
print(sfs1.k_score_)
print(sfs1.k_feature_idx_)
print(sfs1.k_feature_names_)

In [ ]:
#exhastive feature selection
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestClassifier
efs1=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=3)
efs1=efs1.fit(X_train,y_train)
print(efs1.best_idx_)
print(efs1.best_score_)
print(efs1.best_feature_names_)

In [ ]:
# Principal Component Analysis And Linear Discriminant Analysis
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler, LabelEncoder
for col in X.select_dtypes(include=["object"]).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_scaled, y)

print("Explained variance ratio (PCA):", pca.explained_variance_ratio_)
print("Shape after PCA:", X_pca.shape)
print("Shape after LDA:", X_lda.shape)


In [1]:
import pandas as pd
df3=pd.read_csv("house_price_train.csv")
print(df3)

        Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0        1          60       RL  ...        WD         Normal    208500
1        2          20       RL  ...        WD         Normal    181500
2        3          60       RL  ...        WD         Normal    223500
3        4          70       RL  ...        WD        Abnorml    140000
4        5          60       RL  ...        WD         Normal    250000
...    ...         ...      ...  ...       ...            ...       ...
1455  1456          60       RL  ...        WD         Normal    175000
1456  1457          20       RL  ...        WD         Normal    210000
1457  1458          70       RL  ...        WD         Normal    266500
1458  1459          20       RL  ...        WD         Normal    142125
1459  1460          20       RL  ...        WD         Normal    147500

[1460 rows x 81 columns]


In [2]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df3.columns if df3[col].isnull().sum()>0]
X=df3.drop('SalePrice',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df3['SalePrice']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

4


In [3]:
#quasi-constant features
X = df3.drop('SalePrice', axis=1)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

4
['Street', 'Utilities', 'Condition2', 'PoolArea']


In [4]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

0
[]


In [5]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df3, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')

0


In [6]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df3 = df3.dropna(subset=['SalePrice'])
X = df3.drop('SalePrice', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)

Mutual Information Scores:
 Street_Pave              3.232847
RoofMatl_CompShg         3.178248
Condition2_Norm          3.039581
CentralAir_Y             2.950983
Heating_GasA             2.928889
                           ...   
SaleType_CWD             0.000000
MiscFeature_Othr         0.000000
SaleType_Oth             0.000000
SaleCondition_AdjLand    0.000000
SaleCondition_Family     0.000000
Length: 245, dtype: float64

Top 10 Features:
 Index(['KitchenAbvGr', 'Street_Pave', 'Condition2_Norm', 'RoofMatl_CompShg',
       'Heating_GasA', 'CentralAir_Y', 'Electrical_SBrkr', 'Functional_Typ',
       'GarageQual_TA', 'SaleType_WD'],
      dtype='object')


In [7]:
#Anova f_regression
from sklearn.feature_selection import f_regression, SelectKBest
import pandas as pd

X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

selector = SelectKBest(score_func=f_regression, k=10)
selector.fit(X_train, y_train)

top_features = X_train.columns[selector.get_support()]
print("\nTop 10 Features (f_regression):\n", top_features)

scores = pd.Series(selector.scores_, index=X_train.columns)
scores = scores.sort_values(ascending=False)
print("\nRegression Feature Scores:\n", scores.head(10))




Top 10 Features (f_regression):
 Index(['OverallQual', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath',
       'TotRmsAbvGrd', 'GarageCars', 'GarageArea', 'ExterQual_TA',
       'KitchenQual_TA'],
      dtype='object')

Regression Feature Scores:
 OverallQual       1628.570495
GrLivArea         1131.879650
GarageCars         758.024880
GarageArea         727.175552
TotalBsmtSF        654.341155
1stFlrSF           605.576313
ExterQual_TA       499.122655
FullBath           497.954582
TotRmsAbvGrd       431.845019
KitchenQual_TA     376.161106
dtype: float64


In [ ]:
#forward selection
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1 = SFS(RandomForestRegressor(n_jobs=-1, random_state=42),
           k_features=5,
           forward=True,
           scoring='r2',
           cv=3)

sfs1 = sfs1.fit(X_train, y_train)

print("Best CV Score (R²):", sfs1.k_score_)
print("Selected Feature Indices:", sfs1.k_feature_idx_)
print("Selected Feature Names:", sfs1.k_feature_names_)


In [ ]:
#Backward Selection
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1 = SFS(RandomForestRegressor(n_jobs=-1, random_state=42),
           k_features=5,
           forward=False,
           scoring='r2',
           cv=3)

sfs1 = sfs1.fit(X_train, y_train)

print("Best CV Score (R²):", sfs1.k_score_)
print("Selected Feature Indices:", sfs1.k_feature_idx_)
print("Selected Feature Names:", sfs1.k_feature_names_)


In [ ]:
#Exhaustive Feature Selection
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestClassifier
efs1=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=3)
efs1=efs1.fit(X_train,y_train)
print(efs1.best_idx_)
print(efs1.best_score_)
print(efs1.best_feature_names_)

In [ ]:
# Principal Component Analysis And Linear Discriminant Analysis
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("train.csv")

X = df.select_dtypes(include=["int64", "float64"]).drop(columns=["Id", "SalePrice"])
y = df["SalePrice"]

X = X.fillna(X.median())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)

y_class = pd.qcut(y, q=3, labels=["Low", "Medium", "High"])
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X_scaled, y_class)

print("Explained variance ratio (PCA):", pca.explained_variance_ratio_)
print("Shape after PCA:", X_pca.shape)
print("Shape after LDA:", X_lda.shape)


In [9]:
#Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_train)
new_pred=model.predict(X_test)
mse=mean_squared_error(y_train,y_pred)
mae=mean_absolute_error(y_train,y_pred)
r2=r2_score(y_train,y_pred)
print("mse is",mse)
print("mean absolute error is",mae)
print("r2_score is",r2)

mse is 316087245.50945115
mean absolute error is 12079.324804579577
r2_score is 0.9481803477598626
